In [ ]:
# Standard Imports
import numpy as np
import matplotlib.pyplot as plt
import csv, datetime, random, sys
import matplotlib.dates as mdates
import pandas as pd

In [ ]:
def MakeDataFromTxt(filepath):
    """
    Helper function to parse text files in IFM proprietary format.
    """
    with open(filepath, 'r') as file:
        reader = csv.reader(file)

        db = dict()                          # Dictionary that will hold k,v pairs of 
        recent_EPC = ''                      # The most recent EPC that was read, and data is being assigned to
        Time_Or_Data = True                  # Time is true, Data is false
        reached_24 = False

        for l in reader:                     # Iterate over entire text file
            line = l[0]                      # Each line only has one token, so we select it
            
            if len(line)==24:                # If it is an EPC
                db[line] = [[],[]]           # Create new k,v pair in database
                recent_EPC = line            # Current EPC to add to is selected
            
            elif line == '[':                # If start bracket, beginning to collect an array of data
                data = []

            elif line == ']':                # End bracket means end of data array
                if len(data)!=0:
                    if Time_Or_Data:
                        db[recent_EPC][0] = data
                        Time_Or_Data = False
                    else:
                        db[recent_EPC][1] = data
                        Time_Or_Data = True
                data = []

            elif len(line)==8:               # If it is a time stamp
                h,m,s = line.split(':')      # Parse timestamp into hours, minutes, seconds

                if int(h)==23:
                    reached_24 = True        # If end of day reached, must move to Day 2

                if reached_24 and int(h) < 12:
                    day=2
                else:
                    day=1

                data.append(datetime.datetime(1970, 2, day, int(h), int(m), int(s))) # Minus 3 for Pacific Time
            
            elif line ==' ':               # If blank space, then we skip to parse next line
                pass

            else:                          # If it is a data point
                try:
                    data.append(float(line))
                except:
                    pass                   # Error catching case just in case, so annoying errors don't crash program
    return db

In [ ]:
t1 = MakeDataFromTxt('tags_3-8-23.txt')

# Combine all data into one dictionary
combined = dict()
for d in [ t1, ]:
    for k,v in d.items():
        if k in combined:                                 # if tag is already in dict
            for i in range(len(v[0])):                    # iterate over all time points
                if v[0][i] in combined[k]:                # if time point is already in dict
                    combined[k][v[0][i]].append(v[1][i])
                else:                                     # if time point is not in dict
                    combined[k][v[0][i]] = [v[1][i]]
        else:             # if tag is not in dict
            combined[k] = dict()
            for i in range(len(v[0])):
                combined[k][v[0][i]] = [v[1][i]]
for k,v in combined.items():
    for k2,v2 in v.items():
        combined[k][k2] = sum(v2)/len(v2) # avg all points taken during the same second

In [ ]:
for k in combined.keys():
    if k[-4:] in [ "7774", "7F8B", "069C", "2F2B" ]:
        print(k)

print("Done")

In [ ]:
# inside_epcs  = [ '1FC1', '96A1', '5448', '7561', '5E01', '30CE' ]
# outside_epcs = [ '67CC', '7D24', '9859', '7D79', '7263', '34D7' ]

# time_inside  = [datetime.datetime(1970, 2, 1, 16, 7, 0) + datetime.timedelta(seconds=i) for i in range(9600) ]
# time_outside = [datetime.datetime(1970, 2, 1, 16, 7, 0) + datetime.timedelta(seconds=i) for i in range(9600) ]

# avg_inside  = [0.0 for i in range(9600)]
# avg_outside = [0.0  for i in range(9600)]
# in_count  = [0 for i in range(9600)]
# out_count = [0 for i in range(9600)]

# for k,v in combined.items():
#     key = k[-4:]
#     if key in inside_epcs:
#         for k2,v2 in v.items():
#             if k2 > datetime.datetime(1970, 2, 1, 16, 7, 0):
#                 avg_inside[time_inside.index(k2)] += v2
#                 in_count[time_inside.index(k2)] += 1
#     elif key in outside_epcs:
#         for k2,v2 in v.items():
#             if k2 > datetime.datetime(1970, 2, 1, 16, 7, 0):
#                 avg_outside[time_outside.index(k2)] += v2
#                 out_count[time_outside.index(k2)] += 1

# # tim_in  = []
# # tim_out = []
# dat_in  = []
# dat_out = []

# for i in range(9600):
#     if in_count[i]>0:
#         dat_in.append(avg_inside[i] / in_count[i])
#     else:
#         dat_in.append(np.NaN)
    
#     if out_count[i]>0:
#         dat_out.append(avg_outside[i] / out_count[i])
#     else:
#         dat_in.append(np.NaN)

# t_in  = []
# t_out = []
# d_in  = []
# d_out = []

# for i in range(160):
#     try:
#         d_in.append(np.nanmean(dat_in[i*60:(i+1)*60]))
#         t_in.append(i)
#     except:
#         pass
#     try:
#         d_out.append(np.nanmean(dat_out[i*60:(i+1)*60]))
#         t_out.append(i)
#     except:
#         pass


# time_in  = []
# time_out = []
# data_in  = []
# data_out = []
# for i in range(160):
#     if not np.isnan(dat_in[i]):
#         time_in.append(t_in[i])
#         data_in.append(d_in[i])
#     if not np.isnan(dat_out[i]):
#         time_out.append(t_out[i])
#         data_out.append(d_out[i])

In [ ]:
# ins, V_in1   = np.polyfit(time_in, data_in, 1, full=False, cov=True)
# a_in, b_in = ins[0], ins[1]
# ins, V_out1 = np.polyfit(time_out, data_out, 1, full=False, cov=True)
# a_out, b_out = ins[0], ins[1]

# print("Inside:  y = {}x + {}".format(a_in, b_in))
# print("Outside: y = {}x + {}".format(a_out, b_out))
# print("\n")

# z_in, V_in2  = np.polyfit(time_in,  data_in,  2, full=False, cov=True)
# z_out, V_out2 = np.polyfit(time_out, data_out, 2, full=False, cov=True)
# print("Inside:  y = {}x^2 + {}x + {}".format(z_in[0], z_in[1], z_in[2]))
# print("Outside: y = {}x^2 + {}x + {}".format(z_out[0], z_out[1], z_out[2]))

In [ ]:
# yhat = a_in * np.array(time_in) + b_in
# ybar = np.sum(data_in)/len(data_in)
# ssreg = np.sum((yhat - ybar)**2.0)         # or sum([ (yihat - ybar)**2 for yihat in yhat])
# sstot = np.sum((data_in - ybar)**2.0)    # or sum([ (yi - ybar)**2 for yi in y])
# print(ssreg/sstot)

# yhat = a_out * np.array(time_out) + b_out
# ybar = np.sum(data_out)/len(data_out)
# ssreg = np.sum((yhat - ybar)**2.0)         # or sum([ (yihat - ybar)**2 for yihat in yhat])
# sstot = np.sum((data_out - ybar)**2.0)    # or sum([ (yi - ybar)**2 for yi in y])
# print(ssreg/sstot)

# yhat = z_in[0] * np.array(time_in)**2 + z_in[1] * np.array(time_in) + z_in[2]
# ybar = np.sum(data_in)/len(data_in)
# ssreg = np.sum((yhat - ybar)**2.0)         # or sum([ (yihat - ybar)**2 for yihat in yhat])
# sstot = np.sum((data_in - ybar)**2.0)    # or sum([ (yi - ybar)**2 for yi in y])
# print(ssreg/sstot)

# yhat = z_out[0] * np.array(time_out)**2 + z_out[1] * np.array(time_out) + z_out[2]
# ybar = np.sum(data_out)/len(data_out)
# ssreg = np.sum((yhat - ybar)**2.0)         # or sum([ (yihat - ybar)**2 for yihat in yhat])
# sstot = np.sum((data_out - ybar)**2.0)    # or sum([ (yi - ybar)**2 for yi in y])
# print(ssreg/sstot)


In [ ]:
list_dfs = list()

# epcs = [ '1FC1','96A1','5448','7561','5E01','30CE','67CC','7D24','9859','7D79','7263','34D7' ]
# inside_epcs = [ '1FC1','96A1','5448', '7561','5E01', '30CE' ]
# outside_epcs = [ '67CC','7D24','9859','7D79', '7263', '34D7' ]

epcs = [ "7774", "7F8B", "069C", "2F2B" ]


for k,v in combined.items():          # For each EPC in all the readings
    # if k[-4:] in epcs:                # If we have the right EPC
        s = sorted( v.items() )
        time, data = [ j[0] for j in s ], [ j[1] for j in s ]

        sdf = pd.DataFrame(
            data = { 'Datetime':time, k[-4:]:data },
            columns=[ 'Datetime', k[-4:] ]
        )
        list_dfs.append(sdf)

all_data = pd.concat([t for t in list_dfs], ignore_index=False)
all_data = all_data.set_index('Datetime')
all_data = all_data.sort_index()
all_data = all_data.groupby(level=0).sum()
all_data = all_data.groupby(all_data.index).sum()
all_data = all_data.replace(0, "-")
all_data.to_excel("test_data_3-7-23.xlsx")

In [ ]:
# PLOTTING CODE
fig, ax = plt.subplots(figsize=(12, 7))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

color = {
    '1FC1':'red',
    '96A1':'blue',
    '5448':'green',
    '7561':'orange',
    '5E01':'purple',
    '30CE':'brown',
    '67CC':'pink',
    '7D24':'#4F4F4F',
    '9859':'olive',
    '7D79':'cyan',
    '7263':'magenta',
    '34D7':'#949400'
}

for k,v in combined.items():          # For each EPC in all the readings
    if k[-4:] in epcs:                # If we have the right EPC
        s = sorted( v.items() )
        time, data = [ j[0] for j in s ], [ j[1] for j in s ]

        ax.plot(time, data, label=k[-4:], color=color[k[-4:]], linewidth=0.95)

ax.legend()